# 2. Modelling

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

### 2.1. Preparing the data for the model

In [4]:
movements = pd.read_csv('bicimad_data.csv')
movements

,Unnamed: 0,_id,user_day_code,idplug_base,user_type,idunplug_base,travel_time,idunplug_station,ageRange,idplug_station,unplug_hourTime,zip_code,date,hour,day_of_week,day_of_week_number
0,0,{'$oid': '60bbf3f21b91311d90d568ad'},b67ea824acf706087d5818ebc6f90821a5abaab17a0868...,15,1,13,319,38,0,54,2021-06-01 00:00:00+00:00,NaN,2021-06-01,00:00:00,Tuesday,1
1,1,{'$oid': '60bbf3f21b91311d90d568b2'},364b1b99f1f9a1748f7abf1134c8d6a46197b583649308...,7,1,1,386,185,0,126,2021-06-01 00:00:00+00:00,NaN,2021-06-01,00:00:00,Tuesday,1
2,2,{'$oid': '60bbf3f21b91311d90d568b3'},5cc17031cba02835dda4b6c153b56572beaade54d8ab91...,14,1,10,465,15,0,91,2021-06-01 00:00:00+00:00,NaN,2021-06-01,00:00:00,Tuesday,1
3,3,{'$oid': '60bbf3f21b91311d90d568b4'},6e855acaaa17d6d14236598c0f1172c783e16950e39c01...,8,1,22,613,84,0,236,2021-06-01 00:00:00+00:00,NaN,2021-06-01,00:00:00,Tuesday,1
4,4,{'$oid': '60bbf3f21b91311d90d568b5'},e93417502d55208995a61f4177956ab9289aba39044ce3...,1,1,4,751,131,5,94,2021-06-01 00:00:00+00:00,28006,2021-06-01,00:00:00,Tuesday,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397434,413359,{'$oid': '60e3812c1b9131285030f696'},36d40b49fafe4e7f41919089b609debf52f1ee42f3e752...,14,1,20,841,132,5,265,2021-06-30 23:00:00+00:00,28020,2021-06-30,23:00:00,Wednesday,2
397435,413360,{'$oid': '60e381341b9131285030fefb'},04ea09adba4c3e5e9e9142362b0fca2187530f31834001...,23,1,9,1480,182,5,265,2021-06-30 21:00:00+00:00,28005,2021-06-30,21:00:00,Wednesday,2
397436,413361,{'$oid': '60e381351b9131285030ff15'},d26c47762f73fbdca3fefb34830f72f2df1538f9926c73...,22,1,3,762,15,5,265,2021-06-29 18:00:00+00:00,28791,2021-06-29,18:00:00,Tuesday,1
397437,413365,{'$oid': '60e4d2a51b91313ed4f52e62'},45de608bddfda3f468ef6093c65f6246a2917ce2376e98...,23,1,3,746,234,0,84,2021-06-27 19:00:00+00:00,NaN,2021-06-27,19:00:00,Sunday,6


### 2.2. Description of the model

We want to made a model in order to predict the number of trips per hour and stations. We are going to use ML supervised models

#### 2.2.1. Target variable

The target variable are the number of rides per hour and station.

In [20]:
target = pd.DataFrame(movements.groupby(['idunplug_station','date','hour'])['_id'].count())
target

_id
idunplug_station date       hour         
1                2021-06-01 07:00:00    2
                            08:00:00    1
                            10:00:00    1
                            11:00:00    4
                            12:00:00    6
...                                   ...
270              2021-06-30 18:00:00    4
                            19:00:00    2
                            20:00:00    2
                            21:00:00    1
                            23:00:00    1

[123650 rows x 1 columns]

In [21]:
#We merge the dataframe movements with the target to start with our model

In [24]:
movements_model = movements.merge(target, how='inner', left_on=['date', 'idunplug_station', 'hour'], 
                right_on = ['date', 'idunplug_station', 'hour'])
movements_model.describe()

,Unnamed: 0,idplug_base,user_type,idunplug_base,travel_time,idunplug_station,ageRange,idplug_station,day_of_week_number,_id_y
count,397439.000000,397439.000000,397439.000000,397439.000000,397439.000000,397439.000000,397439.000000,397439.000000,397439.000000,397439.000000
mean,207397.663926,12.391177,1.065587,12.366964,952.680283,122.504477,1.890539,122.660559,2.669184,5.096266
std,119417.884300,7.584029,0.551467,7.564775,968.117759,74.388739,2.204560,74.719298,1.847248,3.208265
min,0.000000,1.000000,1.000000,1.000000,60.000000,1.000000,0.000000,1.000000,0.000000,1.000000
25%,104151.500000,5.000000,1.000000,5.000000,499.000000,56.000000,0.000000,56.000000,1.000000,3.000000
50%,207913.000000,12.000000,1.000000,12.000000,744.000000,121.000000,0.000000,121.000000,2.000000,4.000000
75%,310857.500000,19.000000,1.000000,19.000000,1087.000000,184.000000,4.000000,185.000000,4.000000,7.000000
max,413369.000000,30.000000,7.000000,30.000000,14362.000000,270.000000,6.000000,2009.000000,6.000000,27.000000


In [25]:
movements_model.head()

,Unnamed: 0,_id_x,user_day_code,idplug_base,user_type,idunplug_base,travel_time,idunplug_station,ageRange,idplug_station,unplug_hourTime,zip_code,date,hour,day_of_week,day_of_week_number,_id_y
0,0,{'$oid': '60bbf3f21b91311d90d568ad'},b67ea824acf706087d5818ebc6f90821a5abaab17a0868...,15,1,13,319,38,0,54,2021-06-01 00:00:00+00:00,NaN,2021-06-01,00:00:00,Tuesday,1,1
1,1,{'$oid': '60bbf3f21b91311d90d568b2'},364b1b99f1f9a1748f7abf1134c8d6a46197b583649308...,7,1,1,386,185,0,126,2021-06-01 00:00:00+00:00,NaN,2021-06-01,00:00:00,Tuesday,1,1
2,2,{'$oid': '60bbf3f21b91311d90d568b3'},5cc17031cba02835dda4b6c153b56572beaade54d8ab91...,14,1,10,465,15,0,91,2021-06-01 00:00:00+00:00,NaN,2021-06-01,00:00:00,Tuesday,1,2
3,8,{'$oid': '60bbf3f31b91311d90d568ba'},c5d58832c8f4cba6b6beb457b404233fa17e8053ab1483...,15,1,20,397,15,0,175,2021-06-01 00:00:00+00:00,NaN,2021-06-01,00:00:00,Tuesday,1,2
4,3,{'$oid': '60bbf3f21b91311d90d568b4'},6e855acaaa17d6d14236598c0f1172c783e16950e39c01...,8,1,22,613,84,0,236,2021-06-01 00:00:00+00:00,NaN,2021-06-01,00:00:00,Tuesday,1,2


#### 2.2.2. Baseline model

For the first approach, we are going to make a baseline model

In [55]:
movements['hour'] = movements['hour'].str.slice(0,2)

In [56]:
X = movements[['user_type', 'idunplug_base', 'travel_time', 'idunplug_station', 'ageRange', 'hour', 
               'day_of_week_number']]
y = movements_model['_id_y']

In [57]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [58]:
from sklearn.ensemble import RandomForestRegressor
rf= RandomForestRegressor()
rf.fit(X,y)
rf.predict(X)

array([1.2 , 1.26, 1.91, ..., 2.54, 2.01, 2.2 ])